In [1]:
import requests
import pandas as pd
from json.decoder import JSONDecodeError
import json

In [2]:
api_list = [
'3bff9734cfbf4e1aa7b60c9fc224452d',
'ae48b5cb8ffe44c2b061a25785c74d97',
'65e16b9111744ab4840e59166dd659b9',
'4a8e398f23024576ae5e29c505e07270',
'81f3a64644484e32b542e9f48269b489',
'8a7893660af4483ba4da4b18349d6c79',
'9a49f2ddb5134355829ac5ae6946d718',
'e5b1e9fd2dee4f94aa30f7bfe73f21e4',
'0441e2cea6b84a5d976b4c0a037a9e60',
'7251ac941bb64e8cbee0eb58f00883a1',
'e672029cbbc944b586f436bd12a912d7']

In [3]:
companies = pd.read_csv('../datasets/constituents.csv')
company = companies[['Security', 'Symbol']]

In [4]:
ticker = company['Symbol'].to_list()
ticker_dict = {}
for i in ticker:
    ticker_dict[i] = []

In [5]:
ticker2 = company['Security'].to_list()
ticker_dict2 = {}
for i in ticker2:
    ticker_dict2[i] = []

In [6]:
# Get company_news 
df = pd.DataFrame()
empty_list = []
base_url = "https://api.worldnewsapi.com/search-news"
counter = 0
#api_key = "3bff9734cfbf4e1aa7b60c9fc224452d"
for comp in company['Security']:
    try:
        api_key = api_list[counter]
        name = "ORG: " + comp
        params = {
        "language": "en",  
        "api-key": api_key, 
        # very few weekly data
        "earliest-publish-date":"2024-01-01",
        "latest-publish-date" : "2024-04-01",
        'text': name,
        "number":100
        }
        response = requests.get(base_url, params=params)
        data = response.json()
    except JSONDecodeError:
        print(f"took extremely long to get data for {comp}")
        continue
    try:
        df1 = pd.DataFrame(data['news'])
    except KeyError:
        try:
            if (data['code']) == 402:
                if counter == len(api_list) - 1:
                    print(comp)
                    print(api_key)
                    break
                else:
                    counter = counter + 1
                    api_key = api_list[counter]
                    print(f"update api key to {api_key}")
                    params = {
                    "language": "en",  
                    "api-key": api_key, 
                    # very few weekly data
                    "earliest-publish-date":"2024-01-01",
                    "latest-publish-date" : "2024-04-01",
                     'text': name,
                    "number":100
                    }
                    response = requests.get(base_url, params=params)
                    data = response.json()
            else:
                print(data)
        except KeyError:
            print("No news found")
            df1 = pd.DataFrame()

    try:
        ticker_dict2[comp] = df1['id'].to_list()
    except KeyError:
        print("No company data")
        empty_list.append(comp)
        ticker_dict2[comp] = []

    df = pd.concat([df, df1], ignore_index=True)

In [ ]:
df

,id,title,text,url,image,publish_date,author,authors,language,source_country,sentiment
0,175557073,Farmers caught in the growing competition for ...,"Throughout last year (2023), the climate chang...",https://www.agriland.ie/farming-news/farmers-c...,https://cdn.agriland.ie/uploads/2023/12/DSC_24...,2024-01-01 12:00:00,"Jeff Bezos, Justin Roberts","[Jeff Bezos, Justin Roberts]",en,ie,0.149
1,175592149,"Leadership turnover, officer deaths, robbery s...",For Mayor Brandon Johnson and new leaders of t...,https://www.chicagotribune.com/news/criminal-j...,https://www.chicagotribune.com/resizer/XnY9x7C...,2024-01-01 12:00:00,Sam Charles,[Sam Charles],en,us,-0.138
2,175725585,Big workplace changes are coming in 2024. Are ...,If you thought the pandemic and AI revolution ...,https://www.computerworld.com/article/3711762/...,https://images.idgesg.net/images/idge/imported...,2024-01-01 12:00:00,NaN,[],en,us,0.092
3,175559319,Animals Believed to Be Extinct Were Found All ...,The past year brought a glimmer of positive co...,https://www.newsweek.com/animals-believed-exti...,https://d.newsweek.com/en/full/2328624/animals...,2024-01-01 12:00:01,Anna Skinner,[Anna Skinner],en,us,-0.088
4,175585505,Kilkenny photograph is overall junior winner i...,The overall winners of the 2023 Teagasc forest...,https://www.kilkennypeople.ie/news/farming-new...,https://www.kilkennypeople.ie/resizer/-1/-1/tr...,2024-01-01 12:00:01,Sian Moloughney,[Sian Moloughney],en,ie,0.771
...,...,...,...,...,...,...,...,...,...,...,...
195,175577659,"In the Peruvian Andes, the new year is welcome...","The fighting begins and ends with a hug, part ...",https://www.scmp.com/news/world/americas/artic...,https://cdn.i-scmp.com/sites/default/files/d8/...,2024-01-01 14:00:12,Agence France-Presse,[Agence France-Presse],en,cn,0.554
196,175585095,Saskatchewan stories of the year: Experience R...,The troubled organization currently has a $17-...,https://leaderpost.com/news/local-news/saskatc...,https://smartcdn.gprod.postmedia.digital/leade...,2024-01-01 14:00:59,"Angela Amato, Kayle Neis","[Angela Amato, Kayle Neis]",en,ca,-0.116
197,175577749,Every Premier League team's plan and top targe...,"As the calendar turns to 2024, the January tra...",https://www.irishmirror.ie/sport/soccer/premie...,https://i2-prod.irishmirror.ie/incoming/articl...,2024-01-01 14:01:51,Mirror Football,[Mirror Football],en,ie,-0.154
198,175578023,XL Bully campaigners win temporary injunction ...,XL Bully campaigners have won a temporary inju...,https://www.dailymail.co.uk/news/article-12916...,https://i.dailymail.co.uk/1s/2024/01/01/12/795...,2024-01-01 14:03:50,"Dan Grennan, Editor","[Dan Grennan, Editor]",en,gb,-0.356


In [ ]:
df.to_csv("news2.csv", index = False)

In [ ]:
for (key1, value1), (key2, value2) in zip(ticker_dict2.items(), ticker_dict.items()):
    ticker_dict[key2] = value1

In [ ]:
with open("tickers.json", "w") as json_file:
    json.dump(ticker_dict, json_file)

How to read the json:
with open("tickers.json", "r") as json_file:
    ticker_dict = json.load(json_file)